# 一般のロバスト方策勾配の収束について

参考：
* [Policy Gradient in Robust MDPs with Global Convergence Guarantee](https://arxiv.org/abs/2212.10439)

[RL_robust_sa_gradient.ipynb](RL_robust_sa_gradient.ipynb)でやったロバストMDPに対する方策勾配法はSA-rectangularでしか成立しません．
今回は一般のCompactなUncertainty setに対して成り立つ収束について見ていきます．
表記は論文に任せます．

ロバストMDPでは次の最適化問題を解くことを考えます：

$$
\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}):=\boldsymbol{\rho}^{\top} \boldsymbol{v}^{\boldsymbol{\pi}, \boldsymbol{p}}=\sum_{s \in \mathcal{S}} \rho_s v_s^{\boldsymbol{\pi}, \boldsymbol{p}}
$$

また，Occupancy measureを
$$d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}\left(s^{\prime}\right)=(1-\gamma) \sum_{s \in \mathcal{S}} \sum_{t=0}^{\infty} \gamma^t \rho(s) p_{s s^{\prime}}^{\boldsymbol{\pi}}(t)$$
で定義します．
また，簡単のために次の表記も導入しておきます．

$$
\min _{\boldsymbol{\pi} \in \Pi}\left\{\Phi(\boldsymbol{\pi}):=\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} .
$$

あとはこの$\Phi(\pi)$について勾配降下法をすればいい気がしますが，ことはそう単純ではありません．
まず，$\Phi$は微分可能ではなく，さらに凸でも凹でもないので，劣微分も存在しない場合があります（論文では[Non-convex Min-Max Optimization: Applications, Challenges, and Recent Theoretical Advances](https://arxiv.org/abs/2006.08141)あたりを根拠にしていますが，[RL_robust_sa_gradient.ipynb](RL_robust_sa_gradient.ipynb)でみたようにフレシェ劣微分の意味で至るところ微分可能な気がする）．

## Weakly convexとモーロー包

解析を始める前に，いくつか定義を導入しておきます：

---

関数$h: \mathcal{X} \to \mathbb{R}$は次を満たすときに$L$-リプシッツといいます．
任意の$x_1, x_2 \in \mathcal{X}$に対して，$\|h(x_1) - h(x_2)\| \leq L \|x_1 - x_2\|$．

また，任意の$x_1, x_2 \in \mathcal{X}$に対して
$\left\|\nabla h\left(\boldsymbol{x}_1\right)-\nabla h\left(\boldsymbol{x}_2\right)\right\| \leq \ell\left\|\boldsymbol{x}_1-\boldsymbol{x}_2\right\|$のときに$\ell$-smoothといいます．

---

次を満たすときに関数$h: \mathcal{X} \to \mathbb{R}$は$\ell$-weakly convexであるといいます．
任意の$\boldsymbol{g} \in \partial h(\boldsymbol{x})$と $\boldsymbol{x}, \boldsymbol{x}^{\prime} \in \mathcal{X}$について，

$$
h\left(\boldsymbol{x}^{\prime}\right)-h(\boldsymbol{x}) \geq\left\langle\boldsymbol{g}, \boldsymbol{x}^{\prime}-\boldsymbol{x}\right\rangle-\frac{\ell}{2}\left\|\boldsymbol{x}^{\prime}-\boldsymbol{x}\right\|^2
$$

ここで$\partial h(\boldsymbol{x})$はフレシェ劣微分を表します．
ちなみにフレシェ劣微分は次の集合のことです：

$$
\partial h(\boldsymbol{x})=
\left\{u \mid \liminf _{\boldsymbol{x}^{\prime} \rightarrow \boldsymbol{x}} h\left(\boldsymbol{x}^{\prime}\right)-h(\boldsymbol{x})-\left\langle\boldsymbol{u}, \boldsymbol{x}^{\prime}-\boldsymbol{x}\right\rangle /\left\|\boldsymbol{x}^{\prime}-\boldsymbol{x}\right\| \geq 0\right\}
$$

---

$\ell$-smoothな関数は$\ell$-weakly convexです．
証明しましょう．

$r(t):=h\left(x+t\left(x^{\prime}-x\right)\right)$とします．ここで$h(x)=r(0)$かつ$h(x')=r(1)$です．

ここで，

$$
\nabla_t r(t)=\nabla_x h\left(x+t\left(x^{\prime}-x\right)\right)^{\top}\left(x^{\prime}-x\right)
$$

とすると

$$
h\left(x^{\prime}\right)-h(x)=r(1)-r(0)=\int_0^1 \nabla_t r(t) d t
$$
です．
あとは

$$
\begin{aligned}
\| h\left(x^{\prime}\right)-h(x)- & \nabla h(x)^{\top}\left(x^{\prime}-x\right) \| \\
& =\left\|\int_0^1 \nabla r(t) d t-\nabla h(x)^{\top}\left(x^{\prime}-x\right)\right\| \\
& \leq \int_0^1\left\|\nabla r(t)-\nabla h(x)^{\top}\left(x^{\prime}-x\right)\right\| d t \\
& =\int_0^1\left\|\nabla h\left(x+t\left(x^{\prime}-x\right)\right)^{\top}\left(x^{\prime}-x\right)-\nabla h(x)^{\top}\left(x^{\prime}-x\right)\right\| d t \\
& \leq \int_0^1\left\|\nabla h\left(x+t\left(x^{\prime}-x\right)\right)-\nabla h(x)\right\| \cdot\left\|\left(x^{\prime}-x\right)\right\| d t \\
& \leq \int_0^1 t \ell\left\|x^{\prime}-x\right\|^2 d t=\frac{\ell}{2}\left\|x^{\prime}-x\right\|^2 .
\end{aligned}
$$

---


この解析では方策勾配法が停留点（Stationary point）に達したときの解析などをしていきます．
停留点は$0 \in \partial h(x)$で一般に定義されますが，これはnonsmoothな関数ではうまく働かない可能性があります．
これを回避するために，次のモーロー包を考えましょう．

---

関数$h: \mathcal{X} \to \mathbb{R}$ と$\lambda > 0$について，

$$
h_\lambda(x):=\min _{x^{\prime} \in \mathcal{X}}\left\{h\left(x^{\prime}\right)+\frac{1}{2 \lambda}\left\|x-x^{\prime}\right\|^2\right\}
$$

をモーロー包と呼びます．
これは$\lambda$が小さい時$x'\approx x$がminimizerなので，ほぼ$h(x)$と同じ値になります．一方で$lambda$が大きければほぼ$h(x')$の最小値を返すわけですね．

---

In [20]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact

# 適当な関数（非凸）を用意します
x = np.linspace(-0.6, 0.5, 30)
h = x ** 4 - 0.7 * x **2 + 0.1 * x + 1.5 * np.cos(10 * x) * 0.1 + 0.3

def plot_conjugate(lam):
    moreau = np.array([h[i] + (x - x[i]) ** 2 / (2 * lam) for i in range(len(x))]).min(axis=0)
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 1, 1)

    plt.plot(x, h, label=r"$h$")
    plt.plot(x, moreau, label=r"$h_\lambda$")

    plt.grid()
    plt.legend()
    plt.ylim(-0.8, 0.8)
    plt.xlabel("x")

    plt.show()

interact(plot_conjugate, lam=(0.001, 5.0, 0.1))

interactive(children=(FloatSlider(value=2.4010000000000002, description='lam', max=5.0, min=0.001), Output()),…

<function __main__.plot_conjugate(lam)>

このモーロー包を使って停留点を定義しましょう．

---

$\ell$-weakly convex関数$h$について，$x^\star$は次を満たすときに$\epsilon$-first order stationary point ($\epsilon$-FOSP)と呼びます：

$$\left\|\nabla h_{\frac{1}{2 \ell}}\left(x^{\star}\right)\right\| \leq \epsilon$$

ここで$h_{\frac{1}{2 \ell}}(x)$は$\lambda=1 / 2\ell$のモーロー包です．

---

また，モーロー包の勾配は次で与えられます．

---

$h$を$\ell$-weakly convex関数とします．$\lambda < \ell$について，$h_\lambda$は$C^1$-smoothであり，

$$
\nabla h_\lambda(x)=\lambda^{-1}\left(x-\underset{x^{\prime}}{\arg \min }\left(h\left(x^{\prime}\right)+\frac{1}{2 \lambda}\left\|x-x^{\prime}\right\|^2\right)\right)
$$

が成立します．

## ロバスト方策勾配

最悪ケースの遷移$p_t$は
$$
J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right) = \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}\right)
$$
を満たすとします（論文中では誤差がある場合を考えてますが，今回は面倒なので考えません）．

これについて，次の方策更新を考えます．

$$
\begin{aligned}
\boldsymbol{\pi}_{t+1} & \in \arg \min _{\boldsymbol{\pi} \in \Pi}\left\langle\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right), \boldsymbol{\pi}-\boldsymbol{\pi}_t\right\rangle+\frac{1}{2 \alpha_t}\left\|\boldsymbol{\pi}-\boldsymbol{\pi}_t\right\|^2 \\
& =\operatorname{Proj}_{\Pi}\left(\boldsymbol{\pi}_t-\alpha_t \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right)
\end{aligned}
$$

これは各状態について考えると

$$
\boldsymbol{\pi}_{t+1, s}=\operatorname{Proj}_{\Delta A}\left(\boldsymbol{\pi}_{t, s}-\alpha_t \nabla_{\boldsymbol{\pi}_s} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right), \quad \forall s \in \mathcal{S}
$$

として書けます．
$p_t$が固定されたときの勾配は通常のMDPの勾配と同じなので，

$$
\frac{\partial J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})}{\partial \pi_{s a}}=\frac{1}{1-\gamma} \cdot d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}(s) \cdot q_{s a}^{\boldsymbol{\pi}, \boldsymbol{p}} .
$$

で与えられます．

さて，この更新の収束について解析してみましょう．
先に便利な補題を導入します．

### 解析に便利な補題

---

**$J$の性質**

$J_\rho(\boldsymbol{\pi}, \boldsymbol{p})$は$L_\pi$-リプシッツであり，$\ell_\pi$-smoothです．ここで

$$
L_{\boldsymbol{\pi}}:=\frac{\sqrt{A}}{(1-\gamma)^2}, \quad \ell_{\boldsymbol{\pi}}:=\frac{2 \gamma A}{(1-\gamma)^3}
$$

としました．また，$\Phi(\pi)$は$L_\pi$-リプシッツであり，$\ell_\pi$-weakly-convexです．

**証明**　

$\ell_\pi$-smoothの証明は簡単なので飛ばします．
$L_\pi$-リプシッツの証明をしましょう．

$$
\begin{aligned}
\left\|\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\| & =\sqrt{\sum_{s, a}\left(\frac{\partial J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})}{\partial \pi_{s a}}\right)^2} \\
& =\frac{1}{1-\gamma} \sqrt{\sum_a \sum_s\left(d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}(s) q_{s a}^{\boldsymbol{\pi}, \boldsymbol{p}}\right)^2} \\
& \leq \frac{1}{(1-\gamma)^2} \sqrt{\sum_a \sum_s\left(d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{\xi}}(s)\right)^2} \leq \frac{\sqrt{A}}{(1-\gamma)^2},
\end{aligned}
$$
が成立します．これが$L_\pi$リプシッツの証明そのものです．

続いて$\Phi$の証明をしましょう．
$$
\begin{aligned}
\Phi\left(\boldsymbol{\pi}_1\right)-\Phi\left(\boldsymbol{\pi}_2\right) & =\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}\right)-\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}\right) \\
& =J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}_1\right)-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}_2\right) \\
& \leq J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}_1\right)-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}_1\right) \\
& \leq L_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\boldsymbol{\pi}_2\right\| .
\end{aligned}
$$
なので，$\Phi$は$L_\pi$リプシッツです．

$\ell_\pi$-weakly convexの証明は[Efficient Algorithms for Smooth Minimax Optimization](https://arxiv.org/abs/1907.01543)のLemma 3に載っています．

---

[RL_policy_gradient_convergence_rate.ipynb](RL_policy_gradient_convergence_rate.ipynb)でやったように，次のGradient dominanceが成立します：

$$
J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}^{\star}, \boldsymbol{p}\right) \leq \frac{D}{1-\gamma} \max _{\overline{\boldsymbol{\pi}} \in \Pi}(\boldsymbol{\pi}-\overline{\boldsymbol{\pi}})^{\top} \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})
$$

---

$h$を$\ell$-strongly convex functionとします．このとき，任意の$\boldsymbol{x}, \boldsymbol{y} \in \mathcal{X}, h(\boldsymbol{x})$について，

$$
h(\boldsymbol{y})-h(\boldsymbol{x}) \leq \nabla h(\boldsymbol{y})^{\top}(\boldsymbol{y}-\boldsymbol{x})-\frac{\ell}{2}\|\boldsymbol{x}-\boldsymbol{y}\|^2
$$

です．さらに$\boldsymbol{y}=\boldsymbol{x}^{\star}:=\arg \min _{\boldsymbol{x} \in \mathcal{X}} h(\boldsymbol{x})$とすれば，

$$
h(\boldsymbol{x})-\min _{\boldsymbol{x} \in \mathcal{X}} h(\boldsymbol{x}) \geq \frac{\ell}{2}\left\|\boldsymbol{x}-\boldsymbol{x}^{\star}\right\|^2
$$

が成り立ちます．

**証明**

１つ目はStrongly convex functionの定義とほぼ変わりません．また，２つ目は一次の最適性からすぐにわかります．

---


### $\Phi$のGradient dominance

$\Phi$について，次のGradient dominanceが成立します：

$$
\Phi(\boldsymbol{\pi})-\Phi\left(\boldsymbol{\pi}^{\star}\right) \leq\left(\frac{D \sqrt{S A}}{1-\gamma}+\frac{L_{\boldsymbol{\pi}}}{2 \ell_{\boldsymbol{\pi}}}\right)\left\|\nabla \Phi_{\frac{1}{2 \ell_\pi}}(\boldsymbol{\pi})\right\|
$$

ここで$\Phi_\lambda(\pi)$は$\Phi(\pi)$のモーロー包であり，$D:=\sup _{\boldsymbol{\pi} \in \Pi, \boldsymbol{p} \in \mathcal{P}}\left\|\boldsymbol{d}_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}} / \boldsymbol{\rho}\right\|_{\infty}<\infty$としました．

**証明**

$\arg \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})$ の$k$番目の要素を$\boldsymbol{p}^{(k)}$とします．このとき，$J$のGradient dominanceから

$$
\begin{aligned}
\Phi(\boldsymbol{\pi})-\Phi\left(\boldsymbol{\pi}^{\star}\right) & =J\left(\boldsymbol{\pi}, \boldsymbol{p}^{(k)}\right)-J\left(\boldsymbol{\pi}^{\star}, \boldsymbol{p}^{\star}\right) \\
& =J\left(\boldsymbol{\pi}, \boldsymbol{p}^{(k)}\right)-\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J(\boldsymbol{\pi}, \boldsymbol{p}) \\
& \leq J\left(\boldsymbol{\pi}, \boldsymbol{p}^{(k)}\right)-\min _{\boldsymbol{\pi} \in \Pi} J\left(\boldsymbol{\pi}, \boldsymbol{p}^{(k)}\right) \\
& \leq \frac{D}{1-\gamma} \max _{\bar{\pi}}(\boldsymbol{\pi}-\overline{\boldsymbol{\pi}})^{\top} \nabla_{\boldsymbol{\pi}} J\left(\boldsymbol{\pi}, \boldsymbol{p}^{(k)}\right)
\end{aligned}
$$
です．さて，上でやったように，$J_\rho(\pi, p)$は$\pi$について$\ell_\pi$-weakly convexです．
よって，

$$
\tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}):=J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})+\frac{\ell_{\boldsymbol{\pi}}}{2}\|\boldsymbol{\pi}\|^2
$$

が$\pi$について凸であり，$\nabla_{\boldsymbol{\pi}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})=\nabla_{\boldsymbol{\pi}} J_\rho(\boldsymbol{\pi}, \boldsymbol{p})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi}$です．

$\tilde{\Phi(\pi)}:=\max _{\boldsymbol{p} \in \mathcal{P}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})$
とします．$\tilde{J}$の凸性を使えば，Danskinの定理から

$$
\begin{aligned}
\partial \tilde{\Phi}(\boldsymbol{\pi}) & =\operatorname{conv}\left\{\nabla_{\boldsymbol{\pi}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \mid \boldsymbol{p} \in \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} \\
\Longrightarrow \partial \Phi(\boldsymbol{\pi})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi} & =\operatorname{conv}\left\{\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi} \mid \boldsymbol{p} \in \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} \\
\Longrightarrow \partial \Phi(\boldsymbol{\pi}) & =\operatorname{conv}\left\{\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \mid \boldsymbol{p} \in \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} .
\end{aligned}
$$
が成り立ちます(２行目は$\ell_\pi \pi$の部分が$p$に依存しないせいです)

TODO: 続き
